<a href="https://colab.research.google.com/github/MalinRadtke214/Smart-Grid-Research/blob/main/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
%pip install mosaik
%pip install mosaik-battery==3.2.0
%pip install mosaik_components
%pip install mosaik_csv
%pip install import_ipynb

In [2]:
class Model:
    """Simple model that increases its value *val* with some *delta* every
    step.

    You can optionally set the initial value *init_val*. It defaults to ``0``.

    """
    def __init__(self, init_val=0):
        self.val = init_val
        self.delta = 1

    def step(self):
        """Perform a simulation step by adding *delta* to *val*."""
        self.val += self.delta

In [3]:
import mosaik_api_v3

META = {
    'type': 'hybrid',
    'models': {
        'ExampleModel': {
            'public': True,
            'params': ['init_val'],
            'attrs': ['delta', 'val'],
            'trigger': ['delta'],
        },
    },
}


class ExampleSim(mosaik_api_v3.Simulator):
    def __init__(self):
        super().__init__(META)
        self.eid_prefix = 'Model_'
        self.entities = {}  # Maps EIDs to model instances/entities
        self.time = 0

    def init(self, sid, time_resolution, eid_prefix=None):
        if float(time_resolution) != 1.:
            raise ValueError('ExampleSim only supports time_resolution=1., but'
                             ' %s was set.' % time_resolution)
        if eid_prefix is not None:
            self.eid_prefix = eid_prefix
        return self.meta

    def create(self, num, model, init_val):
        next_eid = len(self.entities)
        entities = []

        for i in range(next_eid, next_eid + num):
            model_instance = Model(init_val)
            eid = '%s%d' % (self.eid_prefix, i)
            self.entities[eid] = model_instance
            entities.append({'eid': eid, 'type': model})

        return entities


    def step(self, time, inputs, max_advance):
        self.time = time
        # Check for new delta and do step for each model instance:
        for eid, model_instance in self.entities.items():
            if eid in inputs:
                attrs = inputs[eid]
                for attr, values in attrs.items():
                    new_delta = sum(values.values())
                model_instance.delta = new_delta

            model_instance.step()

        return time + 1  # Step size is 1 second

    def get_data(self, outputs):
        data = {}
        for eid, attrs in outputs.items():
            model = self.entities[eid]
            data['time'] = self.time
            data[eid] = {}
            for attr in attrs:
                if attr not in self.meta['models']['ExampleModel']['attrs']:
                    raise ValueError('Unknown output attribute: %s' % attr)

                # Get model.val or model.delta:
                data[eid][attr] = getattr(model, attr)

        return data


In [4]:
import mosaik.util

# Only for Jupyter notebooks
import nest_asyncio
import import_ipynb


nest_asyncio.apply()

# Sim config
SIM_CONFIG = {
     'CSV': {
        'python': 'mosaik_csv:CSV',
    }
}
END = 10  # 10 seconds

In [5]:
# Create our simulation world
world = mosaik.World(SIM_CONFIG)

In [6]:
powerplantData = world.start("CSV")
model = examplesim.ExampleModel(init_val=2)



2024-03-21 15:03:38.107 | INFO     | mosaik.scenario:start:280 - Starting "CSV" as "CSV-0" ...


TypeError: CSV.init() missing 2 required positional arguments: 'sim_start' and 'datafile'

In [ ]:
world.run(until=END)